In [ ]:
%pip install llama-index-llms-openai
%pip install llama-index-vector-stores-neo4jvector
%pip install llama-index-embeddings-openai
%pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 5.7 MB/s eta 0:00:00


In [ ]:
!pip install llama-index-readers-file

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 7.4 MB/s eta 0:00:00


In [ ]:
#this code inserts data into existing index, to be used only when needed/first time.
import os
import openai
from llama_index.vector_stores.neo4jvector import Neo4jVectorStore
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import StorageContext

# Set your API key and Neo4j credentials
os.environ["OPENAI_API_KEY"] = "your_api_key"
openai.api_key = os.environ["OPENAI_API_KEY"]

# Neo4j connection details
username = "neo4j"
password = "your_password"
url = "neo4j+s://your_db_id.databases.neo4j.io"

# Set dimensions to 384 and use the correct model
embed_model = OpenAIEmbedding(
    model_name="text-embedding-3-small",  # This model will output 384 dimensions
    dimensions=384,  # Explicitly set dimensions
    api_key=os.environ["OPENAI_API_KEY"]
)

# Initialize the Neo4j vector store
neo4j_vector = Neo4jVectorStore(
    username=username,
    password=password,
    url=url,
    embedding_dimension=384  # Match the embedding dimension
)

try:
    # Load your documents
    documents = SimpleDirectoryReader("/content/data").load_data()

    # Create a storage context
    storage_context = StorageContext.from_defaults(vector_store=neo4j_vector)

    # Build the index with the explicit embedding model
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
        embed_model=embed_model,
        show_progress=True  # Added to show progress during indexing
    )

    # Create a query engine
    query_engine = index.as_query_engine()

    # Execute your query
    response = query_engine.query(
        "1) What is Swami Sivananda's advice to westerners? "
        "2) What is the summary of Vidura Neeti and Ashtavakra Geetha?"
    )

    print("Query Response:")
    print(response)

except Exception as e:
    print(f"An error occurred: {str(e)}")
    # Print more detailed error information
    import traceback
    print("\nFull error traceback:")
    print(traceback.format_exc())

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/112 [00:00<?, ?it/s]

Query Response:
1) Swami Sivananda advises Western readers to carefully read through the entire Bhagavad Gita once and then commence a second reading with selectivity, focusing on philosophical, psychological, ethical, and psychical aspects while passing over elements that may seem particularly Hindu and not acceptable to them.

2) The summary of Vidura Neeti and Ashtavakra Geetha is not provided in the given context.


In [ ]:
# Queyring an exiting index
import os
import openai
from llama_index.vector_stores.neo4jvector import Neo4jVectorStore
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import StorageContext

# Set your API key and Neo4j credentials
os.environ["OPENAI_API_KEY"] = "Your_api_key"
openai.api_key = os.environ["OPENAI_API_KEY"]

# Neo4j connection details
username = "neo4j"
password = "your_password"
url = "neo4j+s://your_db_id.databases.neo4j.io"

# Set up embedding model
embed_model = OpenAIEmbedding(
    model_name="text-embedding-3-small",
    dimensions=384,
    api_key=os.environ["OPENAI_API_KEY"]
)

# Initialize the Neo4j vector store
neo4j_vector = Neo4jVectorStore(
    username=username,
    password=password,
    url=url,
    embedding_dimension=384
)

try:
    # Create storage context from existing vector store
    storage_context = StorageContext.from_defaults(vector_store=neo4j_vector)

    # Create index from existing vector store
    index = VectorStoreIndex.from_vector_store(
        vector_store=neo4j_vector,
        embed_model=embed_model
    )

    # Create a query engine with higher similarity threshold and top_k
    query_engine = index.as_query_engine(
        similarity_top_k=5,  # Increase number of retrieved documents
        similarity_threshold=0.5  # Adjust threshold (0-1) for similarity matching
    )

    # Execute your query
    response = query_engine.query(
        "1) What is Swami Sivananda's advice to westerners? "
        "2) What does Vidura Neeti and Ashtavakra Geetha say?"
    )

    print("Query Response:")
    print(response)

except Exception as e:
    print(f"An error occurred: {str(e)}")
    import traceback
    print("\nFull error traceback:")
    print(traceback.format_exc())

Query Response:
Swami Sivananda advises westerners to study the Bhagavad Gita daily, at least one discourse, so that its great lessons are always fresh in memory. Vidura Neeti emphasizes the importance of righteousness, duty, and wisdom in life, while Ashtavakra Geetha focuses on self-realization, detachment, and the nature of the self.


In [ ]:
# Create a query engine with higher similarity threshold and top_k
query_engine = index.as_query_engine(
    similarity_top_k=5,  # Increase number of retrieved documents
    similarity_threshold=0.5  # Adjust threshold (0-1) for similarity matching
)

# Execute your query
response = query_engine.query(
    "1) What is work without attachment? "
    "2) What does Bhagavad Gita, Vidura Neeti and Ashtavakra Geetha say about it?"
)

print("Query Response:")
print(response)

Query Response:
Work without attachment, as described in the Bhagavad Gita, is performing actions without being attached to the results or fruits of those actions. It emphasizes the importance of carrying out one's duties selflessly and without desire for personal gain. Similarly, Vidura Neeti and Ashtavakra Geetha also stress the significance of performing actions without attachment, focusing on the idea of detachment from the outcomes of one's actions and maintaining a sense of equanimity while engaging in work.


In [ ]:
def ask_question(query, top_k=5, similarity_threshold=0.5, wrap_width=70):
    """
    Function to query the index and format the response nicely

    Parameters:
        query (str): The question you want to ask
        top_k (int): Number of similar documents to retrieve (default: 5)
        similarity_threshold (float): Threshold for similarity matching (0-1) (default: 0.5)
        wrap_width (int): Character width for text wrapping (default: 70)
    """
    # Create query engine
    query_engine = index.as_query_engine(
        similarity_top_k=top_k,
        similarity_threshold=similarity_threshold
    )

    # Get response
    response = query_engine.query(query)

    def wrap_text(text, width=wrap_width):
        """Wrap text to specified width while preserving paragraphs"""
        import textwrap
        paragraphs = text.split('\n')
        wrapped_paragraphs = [textwrap.fill(p.strip(), width=width) for p in paragraphs if p.strip()]
        return '\n\n'.join(wrapped_paragraphs)

    # Format and print response
    print("\nQuery Response:")
    print("=" * wrap_width)
    print(wrap_text(str(response)))
    print("=" * wrap_width)

# Example usage:
# Single question
ask_question("What are the main teachings of the Bhagavad Gita?")

# Multiple questions
ask_question("""
1. What is karma yoga?
2. What is bhakti yoga?
3. What is jnana yoga?
""")

# Specific detailed question
ask_question("Explain the concept of dharma in Hindu philosophy.")

# You can also adjust the parameters
ask_question(
    "What is the role of meditation in spiritual practice?",
    top_k=10,  # Get more similar documents
    similarity_threshold=0.3,  # Lower threshold for more diverse results
    wrap_width=80  # Wider text output
)


Query Response:
The main teachings of the Bhagavad Gita include the importance of
performing one's prescribed duties without attachment, the harmony of
different paths such as Jnana Yoga, Bhakti Yoga, and Karma Yoga, the
concept of self-mastery and inner stability, the idea of oneness
between the individual soul and the Supreme Soul, and the significance
of understanding one's true nature and the workings of the mind.

Query Response:
1. Karma Yoga is the path of selfless action where one performs their
duties without attachment to the results of their actions.

2. Bhakti Yoga is the path of devotion where one worships and adores
the Supreme Being with love and reverence, developing a deep
relationship with Him.

3. Jnana Yoga is the path of knowledge where one seeks to understand
the true nature of reality and the self through self-inquiry and
contemplation.

Query Response:
Dharma in Hindu philosophy refers to righteousness, duty, and moral
law that governs one's conduct and behavio

In [ ]:
# Added bgita.txt to the eixsitng index created using LLM Builder
# Got an error for embed_dim=1536, the one created using LLM builder took 384, will have to check this

In [ ]:
def ask_question(query, top_k=5, similarity_threshold=0.5, wrap_width=70):
    """
    Function to query the index and format the response nicely

    Parameters:
        query (str): The question you want to ask
        top_k (int): Number of similar documents to retrieve (default: 5)
        similarity_threshold (float): Threshold for similarity matching (0-1) (default: 0.5)
        wrap_width (int): Character width for text wrapping (default: 70)
    """
    # Create query engine
    query_engine = index.as_query_engine(
        similarity_top_k=top_k,
        similarity_threshold=similarity_threshold
    )

    # Get response
    response = query_engine.query(query)

    def wrap_text(text, width=wrap_width):
        """Wrap text to specified width while preserving paragraphs"""
        import textwrap
        paragraphs = text.split('\n')
        wrapped_paragraphs = [textwrap.fill(p.strip(), width=width) for p in paragraphs if p.strip()]
        return '\n\n'.join(wrapped_paragraphs)

    # Format and print response
    print("\nQuery Response:")
    print("=" * wrap_width)
    print(wrap_text(str(response)))
    print("=" * wrap_width)

# Example usage:
# Single question
ask_question("What are the main teachings of Indian spiritual texts for working professionals, with specific and detailed references from Bhagavad Gita,Ashtavakra Gita and Vidura Neeti. Quote any relavent verses/passages.")




Query Response:
The main teachings of Indian spiritual texts for working professionals
emphasize the importance of performing one's duties diligently and
selflessly, without attachment to the results. In the Bhagavad Gita,
Lord Krishna advises Arjuna to carry out his prescribed duties without
attachment, as performing actions without attachment leads one towards
the Supreme. This teaching is highlighted in Chapter 3, Verse 19,
where Lord Krishna states, "Therefore, without attachment, constantly
perform action which is duty, for, by performing action without
attachment, man verily reaches the Supreme."

Similarly, the Ashtavakra Gita emphasizes the importance of self-
realization and detachment from worldly desires. In Chapter 1, Verse
3, it is mentioned, "You are not the body, nor is the body yours; you
are not the doer, nor the enjoyer; you are consciousness itself, the
eternal witness."

In Vidura Neeti, the focus is on wisdom, righteousness, and the
importance of moral conduct in 

In [ ]:
def ask_question(query, top_k=5, similarity_threshold=0.5, wrap_width=70):
    """
    Function to query the index and format the response nicely

    Parameters:
        query (str): The question you want to ask
        top_k (int): Number of similar documents to retrieve (default: 5)
        similarity_threshold (float): Threshold for similarity matching (0-1) (default: 0.5)
        wrap_width (int): Character width for text wrapping (default: 70)
    """
    # Create query engine
    query_engine = index.as_query_engine(
        similarity_top_k=top_k,
        similarity_threshold=similarity_threshold
    )

    # Get response
    response = query_engine.query(query)

    def wrap_text(text, width=wrap_width):
        """Wrap text to specified width while preserving paragraphs"""
        import textwrap
        paragraphs = text.split('\n')
        wrapped_paragraphs = [textwrap.fill(p.strip(), width=width) for p in paragraphs if p.strip()]
        return '\n\n'.join(wrapped_paragraphs)

    # Format and print response
    print("\nQuery Response:")
    print("=" * wrap_width)
    print(wrap_text(str(response)))
    print("=" * wrap_width)

# Example usage:
# Single question
ask_question("What are the main concepts of Indian spiritualism for working professionals, with specific and detailed references from Bhagavad Gita,Ashtavakra Gita and Vidura Neeti. Quote any relavent verses/passages.")




Query Response:
The main concepts of Indian spiritualism for working professionals can
be summarized as follows:

1. **Duty and Action**: The Bhagavad Gita emphasizes the importance of
performing one's duty without attachment to the results. Lord Krishna
advises Arjuna to carry out his responsibilities as a warrior without
being swayed by desires or outcomes. This concept is highlighted in
Chapter 2, Verse 47: "You have a right to perform your prescribed
duties, but you are not entitled to the fruits of your actions."

2. **Selflessness and Renunciation**: The Ashtavakra Gita delves into
the idea of selflessness and renunciation of ego. It teaches that true
wisdom lies in letting go of the ego and realizing the self as pure
consciousness. Ashtavakra Gita, Chapter 1, Verse 3 states: "You are
not the body, nor is the body yours; you are awareness itself."

3. **Wisdom and Virtue**: Vidura Neeti emphasizes the importance of
wisdom and virtue in one's conduct. It stresses the significance

I can provide some main concepts of Indian spiritualism relevant to working professionals, with specific references from the Bhagavad Gita and Ashtavakra Gita.  Unfortunately, I don't have access to the text of Vidura Neeti in this context to provide specific quotes.

**Bhagavad Gita:**  The Bhagavad Gita, primarily focuses on the path of *Karma Yoga* (selfless action) and the importance of *Dharma* (duty).  For working professionals, this translates to:

* **Performing one's duties diligently and without attachment to results:**  The Gita emphasizes acting selflessly, focusing on the quality of work rather than the rewards.  This helps manage stress and promotes inner peace.  (While I cannot provide specific verse numbers without access to a specific edition, this concept is central to the entire text).

* **Maintaining equanimity in the face of success and failure:** The Gita teaches the importance of emotional balance, not being overly elated by success or dejected by failure. This is crucial for maintaining a stable and productive work-life balance. (Again, this is a pervasive theme throughout the Gita).

* **Cultivating detachment from outcomes:**  While working hard is essential, the Gita advises against being overly attached to the results of one's efforts. This reduces stress and anxiety associated with performance pressures. (This is a recurring theme, particularly in discussions of Karma Yoga).


**Ashtavakra Gita:** This text emphasizes *Self-Realization* and *Non-Attachment*.  For working professionals, this means:

* **Understanding the true nature of self:** The Ashtavakra Gita encourages introspection and self-awareness, helping professionals understand their motivations and values, leading to more fulfilling careers. (Specific verses would require access to a particular translation).

* **Cultivating detachment from material desires:** The text promotes detachment from material possessions and worldly success, helping professionals avoid burnout and find satisfaction beyond their professional achievements. (This is a core theme throughout the text).

* **Finding inner peace amidst challenges:** The Ashtavakra Gita provides tools for managing stress and finding inner peace, even in demanding work environments.  (Again, specific verses would require access to a particular translation).


In summary, both texts offer valuable insights for working professionals seeking a more balanced and fulfilling life.  They emphasize the importance of mindful action, emotional regulation, and self-awareness in navigating the challenges of the professional world.  To get specific verses, you would need to consult a version of the Bhagavad Gita and Ashtavakra Gita with verse numbering.
